In [ ]:
%load_ext autoreload
%autoreload 2
import os
import time 

from pydrake.systems.framework import DiagramBuilder
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.multibody.parsing import LoadModelDirectives, Parser, ProcessModelDirectives
from pydrake.geometry import Role, RoleAssign
from pydrake.systems.meshcat_visualizer import ConnectMeshcatVisualizer
from pydrake.geometry.optimization import IrisInConfigurationSpace, IrisOptions
from pydrake.planning.common_robotics_utilities import SimpleRRTPlannerState

from comparison.planning import PRM, BiRRT
from comparison.helpers import ForwardKinematics, visualize_trajectory, get_traj_length, make_traj

import numpy as np
import pandas as pd

from meshcat import Visualizer

from spp.linear import LinearSPP


from IPython.display import HTML, SVG
import multiprocessing as mp

from pydrake.solvers.gurobi import GurobiSolver
from pydrake.solvers.mosek import MosekSolver
GurobiSolver.AcquireLicense()
MosekSolver.AcquireLicense()

In [ ]:
# Setup meshcat
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])

# Sporadically need to run `pkill -f meshcat`

In [ ]:
SEED = 17

In [ ]:
vis = Visualizer(zmq_url=zmq_url)
vis.delete()
display(vis.jupyter_cell())

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
parser = Parser(plant)
parser.package_map().Add("mp-gcs", os.path.dirname("./package.xml"))

directives_file = "./models/iiwa14_spheres_collision_welded_gripper.yaml"
directives = LoadModelDirectives(directives_file)
models = ProcessModelDirectives(directives, plant, parser)
[iiwa, wsg, shelf, binR, binL, table] =  models
 

plant.Finalize()

viz_role = Role.kIllustration
# viz_role = Role.kProximity
visualizer = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url,
                                      delete_prefix_on_load=False, role=viz_role)
diagram = builder.Build()

visualizer.load()
context = diagram.CreateDefaultContext()
#plant_context = plant.GetMyContextFromRoot(context)
plant_context = plant.GetMyMutableContextFromRoot(context)
#q0 = [1.3, 0.7, 0.0, -1.6, 0.0, 0.8, 1.41]
#plant.SetPositions(plant_context, q0)
diagram.Publish(context)

# Generate Iris Regions
### via manual seeds

In [ ]:
iris_options = IrisOptions()
iris_options.require_sample_point_is_contained = True
iris_options.iteration_limit = 10
iris_options.termination_threshold = -1
iris_options.relative_termination_threshold = 0.01
iris_options.enable_ibex = False
CORE_CNT = mp.cpu_count() # you may edit this

In [ ]:
#used for paper 
seed_points =  {"Above Shelve": [0, 0.4, 0, -0.8, 0, 0.35, 1.57],   
                "Top Rack":[0, 0.45, 0, -1.35, 0, -0.25, 1.57],   
                "Middle Rack":[0, 0.8, 0, -1.5, 0, -0.7, 1.57],       
                "Left Bin":[1.57, 0.7, 0, -1.6, 0, 0.8, 1.57], 
                "Right Bin":[-1.57, 0.7, 0, -1.6, 0, 0.8, 1.57],
                "Front to Shelve":[0, 0.2, 0, -2.09, 0, -0.3, 1.57], 
                "Left to Shelve":[0.8, 0.7, 0, -1.6, 0, 0, 1.57],
                "Right to Shelve":[-0.8, 0.7, 0, -1.6, 0, 0, 1.57]} 

In [ ]:
#optimized for paper
seed_points = { "Above Shelve": [0, 0.4, 0, -0.8, 0, 0.35, 1.57],       
                "Top Rack": [0, 0.45, 0, -1.35, 0, -0.25, 1.57],    
                "Middle Rack": [0, 0.8, 0, -1.5, 0, -0.7, 1.57],       
                "Left Bin": [1.57, 0.7, 0, -1.6, 0, 0.8, 0],          
                "Right Bin": [-1.57, 0.7, 0, -1.6, 0, 0.8, 0],
                "Helper 1": [0.0, 0.175, 0.0, -1.675, 0.0, -0.275, 1.57],
                "Helper 2":[-0.23670698873773643, 0.0869880191351135,
                                -0.14848147591054467, -1.8845683628360246,
                                    0.11582904486351761, -0.004801857412368382, 1.2343852721657547],
                "Helper 3":[0.1901439226352184, 0.24791935747561322, 0.026105794277764514,
                                -1.862822617783086, 0.2509538957570004, -0.2466088852764272, 1.3435623440091402],
                "Helper 4":[-0.43303849430001307, 0.15450520762404665, 0.30334346818001523,
                                -1.0376976962796667, 0.11739903437607266, 0.5348776947156673, 1.0783084430904017]}

In [ ]:
def calcRegion(seed, verbose):
    start_time = time.time()
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)
    plant.SetPositions(plant_context, seed)
    hpoly = IrisInConfigurationSpace(plant, plant_context, iris_options)
    print("Seed:", seed, "\tTime:", time.time() - start_time, flush=True)
    return hpoly

def generateRegions(seed_points, verbose = True):
    seeds = list(seed_points.values()) if type(seed_points) is dict else seed_points
    regions = []
    loop_time = time.time()
    with mp.Pool(processes = CORE_CNT) as pool:
        regions = pool.starmap(calcRegion, [[seed, verbose] for seed in seeds])
    
    if verbose:
        print("Loop time:", time.time() - loop_time)
    
    if type(seed_points) is dict:
        return dict(list(zip(seed_points.keys(), regions)))
    
    return regions

In [ ]:
regions = generateRegions(seed_points)

In [ ]:
spp = LinearSPP(regions.copy())

In [ ]:
SVG(spp.VisualizeGraph()) 

# Build PRM

In [ ]:
collision_step_size = 0.02
K = 5
roadmap_size = 15000

#initlize with seeds for fairness
prm = PRM(plant, diagram, collision_step_size, SEED, K, list(seed_points.values()))

def weighted_distance_function(q1,q2):
    "Weights used by TRI for iiwa"
    weights = np.array([2.0, 2.0, 2.0, 2.0, 1.0, 1.0, 1.0])
    return np.sqrt(np.square(q1-q2).dot(weights))

prm.distance_fn = weighted_distance_function 

### Load Roadmap if you don't want to wait 15 minutes

In [ ]:
#TODO use wget here to download th prm map from server
prm.load("15k_presplined.rmp")

### Or Grow it 

In [ ]:
stats = prm.GrowRoadMap(roadmap_size, True)
print(f'Grow time: {stats["growing_time"]} s') 

### Connect Narrow Passages with BiRRT to refine roadmap
PRMs sampling is not dense enough to connect racks of the shelve. So we resort to using BiRRT to find paths through knwon narrow passages. Then we add the waypoints along the path to the prm roadmap.

In [ ]:
solve_timeout = 200
goal_bias = 0.05
step_size = 0.1
birrt = BiRRT(plant, diagram, collision_step_size, SEED, step_size, goal_bias, solve_timeout)
birrt.distance_fn = weighted_distance_function

In [ ]:
def include_subtree_fn(node_val):
    return list(node_val) in list(seed_points.values())

subgraphs, subtrees = prm.SubgraphsIdxAndTrees(include_subtree_fn)
print(f"Found {len(subgraphs)} disjoint subgraphs of interest")

In [ ]:
start_time = time.time()

main_graph, main_tree = subgraphs[0], subtrees[0]

for secondary_tree in subtrees[1:]:
    result, _, _ = birrt.connect(main_tree, secondary_tree, use_parallel = True)
    #prm.addNodes(result.Path())

print(f"Roadmap extended to {prm.roadmap_size} in {round(time.time()-start_time,2)} s")

### Approach 2: Connect via RRT first and then grow roadmap

In [ ]:
pairs_2_connect = [
  [SimpleRRTPlannerState(seed_points["Left Bin"]), SimpleRRTPlannerState(seed_points["Above Shelve"])],
  [SimpleRRTPlannerState(seed_points["Left Bin"]), SimpleRRTPlannerState(seed_points["Left to Shelve"])],
  [SimpleRRTPlannerState(seed_points["Left Bin"]), SimpleRRTPlannerState(seed_points["Front to Shelve"])],
    
  [SimpleRRTPlannerState(seed_points["Right Bin"]), SimpleRRTPlannerState(seed_points["Above Shelve"])],
  [SimpleRRTPlannerState(seed_points["Right Bin"]), SimpleRRTPlannerState(seed_points["Right to Shelve"])],
  [SimpleRRTPlannerState(seed_points["Right Bin"]), SimpleRRTPlannerState(seed_points["Front to Shelve"])],
    
  [SimpleRRTPlannerState(seed_points["Above Shelve"]), SimpleRRTPlannerState(seed_points["Left to Shelve"])],
  [SimpleRRTPlannerState(seed_points["Above Shelve"]), SimpleRRTPlannerState(seed_points["Right to Shelve"])],
    
  [SimpleRRTPlannerState(seed_points["Front to Shelve"]), SimpleRRTPlannerState(seed_points["Top Rack"])],
  [SimpleRRTPlannerState(seed_points["Front to Shelve"]), SimpleRRTPlannerState(seed_points["Middle Rack"])],

]


#initlize with seeds for fairness
prm = PRM(plant, diagram, collision_step_size, SEED, K)
prm.distance_fn = weighted_distance_function 

start_time = time.time()

for start_tree, goal_tree in  pairs_2_connect:
    result, start_tree_extended, end_tree_extended = birrt.connect([start_tree], [goal_tree], use_parallel = True)
    if len(result.Path()) == 0:
        raise Exception("Failed to connect nodes")
    prm.addNodes(list(map(lambda x: x.GetValueImmutable(), start_tree_extended)) + list(map(lambda x: x.GetValueImmutable(), end_tree_extended)))

print(f"Roadmap extended to {prm2.roadmap_size} in {round(time.time()-start_time,2)} s")

In [ ]:
## continue growing roadmap
stats = prm.GrowRoadMap(roadmap_size, True)
print(f'Grow time: {stats["growing_time"]} s') 

# Run Comparison

In [ ]:
def getSPPPath(sequence, verbose = False):

    path = [sequence[0]]
    run_time = 0.0
    for start_pt, goal_pt in zip(sequence[:-1], sequence[1:]):
        spp = LinearSPP(regions.copy())
        start_time = time.time()
        waypoints, result_relax, best_result_relax, hard_result_relax = spp.SolvePath(start_pt, goal_pt,
                                                                                       rounding=True,
                                                                                       verbose=False,
                                                                                       preprocessing = True)
        if waypoints is None:
            if verbose:
                print(f"Failed between {start_pt} and {goal_pt}")
            return None

        
        run_time += result_relax.get_solver_details().optimizer_time
        max_hard_result_time = 0
        for result in hard_result_relax:
            if result.get_solver_details().optimizer_time > max_hard_result_time:
                max_hard_result_time = result.get_solver_details().optimizer_time
        run_time += max_hard_result_time
        
        path += waypoints.T[1:].tolist()
    
    return np.stack(path).T, run_time

def shortcut(path):
    """Those values are used by TRI"""
    return prm.shortcut(np.stack(path),
                 max_iter = 200,
                 max_failed_iter = 200,
                 max_backtracking_steps = 0,
                 max_shortcut_fraction = 1.0,
                 resample_shortcuts_interval = 0.25,
                 check_for_marginal_shortcuts = False)

# Demonstration

In [ ]:
demo_a = [seed_points["Above Shelve"],
          seed_points["Top Rack"]]

demo_b = [seed_points["Top Rack"],
          seed_points["Middle Rack"]]

demo_c = [seed_points["Middle Rack"],
          seed_points["Left Bin"]]

demo_d = [seed_points["Left Bin"],
          seed_points["Right Bin"]]

demo_e = [seed_points["Right Bin"],
          seed_points["Above Shelve"]]


demo_circle = [seed_points["Above Shelve"],
               seed_points["Top Rack"],
               seed_points["Middle Rack"],
               seed_points["Left Bin"],
               seed_points["Right Bin"],
               seed_points["Above Shelve"]]

##pertubated 
#used for paper 
#PRM fails to connect the following milestons
milestones_pertubated =  {"Above Shelve": [0.26, 0.4, 0, -0.8, 0, 0.35, 1.57],   
                "Top Rack":[-0.16, 0.4, 0.0, -1.41, 0.0, -0.26, 0.0],   
                "Middle Rack":[-0.21, 0.75, 0.0, -1.57, 0.0, -0.69, 0],       
                "Left Bin":[1.31, 0.7, 0.0, -1.6, 0.0, 0.8, 0.88], 
                "Right Bin":[-1.28, 0.7, 0.0, -1.6, 0.0, 0.8, 0.3]}  
        
#This was the best PRM could do
milestones_pertubated =  {"Above Shelve": [-0.26, 0.4, 0, -0.8, 0, 0.35, 1.57],   
                "Top Rack":[0.16, 0.4, 0.0, -1.41, 0.0, -0.26, 0.4],   
                "Middle Rack":[0.21, 0.75, 0.0, -1.57, 0.0, -0.69, 0],       
                "Left Bin":[1.3, 0.7, 0.0, -1.6, 0.0, 0.8, 1.41], 
                "Right Bin":[-1.46, 0.7, 0.0, -1.6, 0.0, 0.8, 0.37]} 
                         

demo_a_pert = [milestones_pertubated["Above Shelve"],
          milestones_pertubated["Top Rack"]]

demo_b_pert = [milestones_pertubated["Top Rack"],
          milestones_pertubated["Middle Rack"]]

demo_c_pert = [milestones_pertubated["Middle Rack"],
          milestones_pertubated["Left Bin"]]

demo_d_pert = [milestones_pertubated["Left Bin"],
          milestones_pertubated["Right Bin"]]

demo_e_pert = [milestones_pertubated["Right Bin"],
          milestones_pertubated["Above Shelve"]]


demo_circle_pert = [milestones_pertubated["Above Shelve"],
               milestones_pertubated["Top Rack"],
               milestones_pertubated["Middle Rack"],
               milestones_pertubated["Left Bin"],
               milestones_pertubated["Right Bin"],
               milestones_pertubated["Above Shelve"]]


In [ ]:
execute_demo = demo_circle_pert
SPP_path, SPP_time = getSPPPath(execute_demo, verbose = False)
linear_spp_traj = make_traj(SPP_path, speed = 2)
print(f"Linear SPP length: {round(get_traj_length(linear_spp_traj), 3)}, and time: {round(SPP_time, 3)} s")

PRM_path, PRM_time = prm.getPath(execute_demo, verbose = True)
prm_traj = make_traj(PRM_path, speed = 2)
print(f"PRM length: {round(get_traj_length(prm_traj), 3)}, and time: {round(PRM_time, 3)} s")

sPRM_path, sPRM_time = prm.getPath(execute_demo, path_processing = shortcut)
sprm_traj = make_traj(sPRM_path, speed = 2)
print(f"Smoothed PRM length: {round(get_traj_length(sprm_traj), 3)}, and time: {round(sPRM_time, 3)} s")


In [ ]:
vis_meshcat = visualize_trajectory(zmq_url,
                    [linear_spp_traj,prm_traj,sprm_traj],
                     show_line = True,
                     iiwa_ghosts = execute_demo,
                     alpha =  0.3,
                     regions = [])


In [ ]:
with open ("SPP_PRM15k_comparison.html", "w") as f:
    f.write(vis_meshcat.static_html())

## Comparison Table for Paper


In [ ]:
#demos_for_paper = [demo_a, demo_b, demo_c, demo_d, demo_e]
demos_for_paper = [demo_a_pert, demo_b_pert, demo_c_pert, demo_d_pert, demo_e_pert] #slight pertubations
#extract trajectory and time 
data = map(lambda d:[getSPPPath(d),prm.getPath(d),prm.getPath(d, path_processing = shortcut)], demos_for_paper)
#data = map(lambda d:[prm.getPath(d),prm.getPath(d, path_processing = shortcut)], demos_for_paper)
#get length of trajectory
data = map(lambda traj_time: [[get_traj_length(make_traj(trj)), t] for trj, t in traj_time], data)
#flatten for pandas
data = np.array(list(data))

In [ ]:
cols = {"Proposed Planner":data[:,0].flatten(),"Regular PRM":data[:,1].flatten(),"Shortcut PRM":data[:,2].flatten()}

index = pd.MultiIndex.from_tuples([('q1-q2', 'length (rad)'), ('q1-q2', 'runtime (s)'),
                                    ('q2-q3', 'length (rad)'), ('q2-q3', 'runtime (s)'),
                                    ('q3-q4', 'length (rad)'), ('q3-q4', 'runtime (s)'),
                                    ('q4-q5', 'length (rad)'), ('q4-q5', 'runtime (s)'),
                                    ('q5-q1', 'length (rad)'), ('q5-q1', 'runtime (s)'),], names=["Task", ""])
df = pd.DataFrame(data = cols, index=index )

In [ ]:
df.round(2)